Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [9]:
batch_size = 128
hidden_nodes = 1024
reg_coef = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    
  # for the hidden layer
  h_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  h_biases = tf.Variable(tf.zeros([hidden_nodes]))
  # shape: (batch_size, 768) x (768, 1024) => (batch_size, 1024)
  h_output = tf.matmul(tf_train_dataset, h_weights) + h_biases
    
  # do the ReLU
  relu_output = tf.nn.relu(h_output)

  # for the 2nd layer, which generates the labels
  r_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  r_biases = tf.Variable(tf.zeros([num_labels]))
  # shape: (batch_size, 1024) x (1024, 10) => (batch_size, 10)
  logits = tf.matmul(relu_output, r_weights) + r_biases
    
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  l2_regularization = reg_coef * tf.nn.l2_loss(h_weights) + reg_coef * tf.nn.l2_loss(r_weights)
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss + l2_regularization)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )
  test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )

# run it. no code change compared to the previous task

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 334.192810
Minibatch accuracy: 14.1%
Validation accuracy: 23.8%
Minibatch loss at step 500: 0.434767
Minibatch accuracy: 85.9%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.615258
Minibatch accuracy: 82.0%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 0.399264
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.437798
Minibatch accuracy: 90.6%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.540044
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 0.595008
Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Test accuracy: 90.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

__Daniel's Notes__: Using small datasets and no regularization, training accuracy is 100% (overfitting), test accuracy is 72%. Using regularization, prediction accuracy is still 100%, but test accuracy is now 76.7%

---

In [11]:
batch_size = 128
hidden_nodes = 1024
reg_coef = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    
  # for the hidden layer
  h_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  h_biases = tf.Variable(tf.zeros([hidden_nodes]))
  # shape: (batch_size, 768) x (768, 1024) => (batch_size, 1024)
  h_output = tf.matmul(tf_train_dataset, h_weights) + h_biases
    
  # do the ReLU
  relu_output = tf.nn.relu(h_output)

  # for the 2nd layer, which generates the labels
  r_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  r_biases = tf.Variable(tf.zeros([num_labels]))
  # shape: (batch_size, 1024) x (1024, 10) => (batch_size, 10)
  logits = tf.matmul(relu_output, r_weights) + r_biases
    
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  l2_regularization = reg_coef * tf.nn.l2_loss(h_weights) + reg_coef * tf.nn.l2_loss(r_weights)
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss + l2_regularization)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )
  test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )

# run it. no code change compared to the previous task

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # always use offset=0 to limit the size of datasets
    offset = 0
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 418.582825
Minibatch accuracy: 12.5%
Validation accuracy: 26.7%
Minibatch loss at step 500: 0.017898
Minibatch accuracy: 100.0%
Validation accuracy: 67.2%
Minibatch loss at step 1000: 0.040001
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 1500: 0.037341
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 2000: 0.036319
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 2500: 0.035787
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 3000: 0.035448
Minibatch accuracy: 100.0%
Validation accuracy: 69.8%
Test accuracy: 76.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

__Daniel's Notes__:
* Don't use dropout on training data => that would throw away lots of useful data.
* Use dropout on hidden layers => increase test accuracy to 77.4% (w/ regularization on extreme overfitting case), although prediction accuracy is still 100% (overfitting)

---

In [15]:
batch_size = 128
hidden_nodes = 1024
reg_coef = 0.01
dropout_keep = 0.5

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    
  # for the hidden layer
  h_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  h_biases = tf.Variable(tf.zeros([hidden_nodes]))
  # shape: (batch_size, 768) x (768, 1024) => (batch_size, 1024)
  h_output = tf.matmul(tf_train_dataset, h_weights) + h_biases
    
  # do the ReLU
  relu_output = tf.nn.relu(h_output)

  # for the 2nd layer, which generates the labels
  r_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  r_biases = tf.Variable(tf.zeros([num_labels]))
  # shape: (batch_size, 1024) x (1024, 10) => (batch_size, 10)
  logits = tf.matmul(tf.nn.dropout(relu_output, dropout_keep), r_weights) + r_biases
    
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  l2_regularization = reg_coef * tf.nn.l2_loss(h_weights) + reg_coef * tf.nn.l2_loss(r_weights)
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss + l2_regularization)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )
  test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )

# run it. no code change compared to the previous task

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # always use offset=0 to limit the size of datasets
    offset = 0
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 543.560913
Minibatch accuracy: 11.7%
Validation accuracy: 26.8%
Minibatch loss at step 500: 0.013815
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 1000: 0.045080
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 1500: 0.040294
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 2000: 0.039165
Minibatch accuracy: 100.0%
Validation accuracy: 69.8%
Minibatch loss at step 2500: 0.040861
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Minibatch loss at step 3000: 0.039438
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Test accuracy: 77.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
__Daniel's Notes__:
* We can indeed wrap reusable code in a single function
* Seems that adding an extra layer will make gradient to unstable: http://stackoverflow.com/questions/36565430/adding-multiple-layers-to-tensorflow-causes-loss-function-to-become-nan
  - Reduce learning rate from 0.5 to 0.01 helps a lot. with learning_rate=0.5, loss actually increases exponentially to NaN.
  - Could try Xavier initialization: https://www.tensorflow.org/versions/r0.11/api_docs/python/contrib.layers/initializers#xavier_initializer
  - Could try learning rate decay. But since we already use small learning_rate, it might not be very useful
* Useful resource: http://www.ritchieng.com/machine-learning/deep-learning/tensorflow/regularization/
  - 5 hidden layers, 50% less for each layer
  - overfitting prevention: l2 regularization, dropout
  - test accuracy: 93.4%
* Adding an extra layer but not training enough does not help too much. It might be enough just to use 1 layer.
* More optimization would be left to HPO
 
 ---


In [19]:
batch_size = 128
hidden_nodes = [1024, 256]
reg_coef = 0.01
dropout_keep = 0.5
learning_rate = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # for the hidden layer
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes[0]]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes[0]]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes[0], hidden_nodes[1]]))
  biases_2 = tf.Variable(tf.zeros([hidden_nodes[1]]))
  r_weights = tf.Variable(tf.truncated_normal([hidden_nodes[1], num_labels]))
  r_biases = tf.Variable(tf.zeros([num_labels]))

  # verify that each new local variable here uses a different "variable" name in tf.
  def get_nn(dataset, dropout=False):
    output_1 = tf.matmul(dataset, weights_1) + biases_1
    output_1_relu = tf.nn.relu(output_1)
    if dropout:
      output_1_final = tf.nn.dropout(output_1_relu, dropout_keep)
    else:
      output_1_final = output_1_relu
    output_2 = tf.matmul(output_1_final, weights_2) + biases_2
    output_2_relu = tf.nn.relu(output_2)
    if dropout:
      output_2_final = tf.nn.dropout(output_2_relu, dropout_keep)
    else:
      output_2_final = output_2_relu
    return tf.matmul(output_2_final, r_weights) + r_biases

  logits = get_nn(tf_train_dataset, dropout=False)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  l2_regularization = reg_coef * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(r_weights))

  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss + l2_regularization)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax(get_nn(tf_valid_dataset))
  test_prediction = tf.nn.softmax(get_nn(tf_test_dataset))

# run it. no code change compared to the previous task

num_steps = 15001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3218.383789
Minibatch accuracy: 16.4%
Validation accuracy: 22.9%
Minibatch loss at step 500: 17.897339
Minibatch accuracy: 68.0%
Validation accuracy: 66.2%
Minibatch loss at step 1000: 7.088058
Minibatch accuracy: 61.7%
Validation accuracy: 58.5%
Minibatch loss at step 1500: 2.758253
Minibatch accuracy: 59.4%
Validation accuracy: 54.8%
Minibatch loss at step 2000: 1.780552
Minibatch accuracy: 64.8%
Validation accuracy: 59.1%
Minibatch loss at step 2500: 2.161883
Minibatch accuracy: 55.5%
Validation accuracy: 60.8%
Minibatch loss at step 3000: 1.617876
Minibatch accuracy: 61.7%
Validation accuracy: 62.9%
Minibatch loss at step 3500: 1.236932
Minibatch accuracy: 64.1%
Validation accuracy: 65.1%
Minibatch loss at step 4000: 1.185094
Minibatch accuracy: 70.3%
Validation accuracy: 67.2%
Minibatch loss at step 4500: 0.990069
Minibatch accuracy: 68.0%
Validation accuracy: 67.9%
Minibatch loss at step 5000: 0.856811
Minibatch accuracy: 69.5%
Validation acc